In [1]:
import datetime as dt
import time
import random
import logging

from optibook.synchronous_client import Exchange

exchange = Exchange()
exchange.connect()

logging.getLogger('client').setLevel('ERROR')

2021-11-26 21:44:52,271 [asyncio   ] [MainThread  ] Using selector: EpollSelector
2021-11-26 21:44:52,273 [client    ] [Thread-4    ] background thread started
2021-11-26 21:44:52,281 [client    ] [Thread-4    ] opened connection
2021-11-26 21:44:52,281 [client    ] [Thread-4    ] start read <StreamReader t=<_SelectorSocketTransport fd=95 read=polling write=<idle, bufsize=0>>>
2021-11-26 21:44:52,283 [client    ] [Thread-4    ] logged in!


In [2]:
def trade_would_breach_position_limit(instrument_id, volume, side, position_limit):
    positions = exchange.get_positions()
    position_instrument = positions[instrument_id]

    if side == 'bid':
        return position_instrument + volume > position_limit
    elif side == 'ask':
        return position_instrument - volume < -position_limit
    else:
        raise Exception(f'''Invalid side provided: {side}, expecting 'bid' or 'ask'.''')

In [13]:
def print_positions_and_pnl():
    positions = exchange.get_positions()
    pnl = exchange.get_pnl()

    print('Positions:')
    for instrument_id in positions:
        print(f'  {instrument_id:10s}: {positions[instrument_id]:4.0f}')

    print(f'\nPnL: {pnl:.2f}')
print(exchange.get_positions())
print(exchange.get_pnl())

{'PHILIPS_A': -1, 'PHILIPS_B': 0}
17.200000000000003


2021-11-26 22:47:16,003 [client    ] [Thread-4    ] Forcing a disconnect due to an error: Closing connection because someone else logged in with the same credentials. Only one session may be active at the same time.


In [ ]:
STOCK_A_ID = 'PHILIPS_A'
STOCK_B_ID = 'PHILIPS_B'

while True:
    print(f'')
    print(f'-----------------------------------------------------------------')
    print(f'TRADE LOOP ITERATION ENTERED AT {str(dt.datetime.now()):18s} UTC.')
    print(f'-----------------------------------------------------------------')

    print_positions_and_pnl()
    print(f'')
    
    while True:
        if abs(exchange.get_last_price_book(STOCK_A_ID).asks[0].price - exchange.get_last_price_book(STOCK_A_ID).bids[0].price) > abs(exchange.get_last_price_book(STOCK_B_ID).asks[0].price - exchange.get_last_price_book(STOCK_B_ID).bids[0].price):
            stock_id = STOCK_A_ID
            bid=bida
            ask=aska
        else:
            stock_id = STOCK_B_ID
            bid=bidb
            ask=aska

        if exchange.get_positions()[stock_id] > 0:
            if bid < exchange.get_last_price_book(stock_id).asks[0].price:
                side = 'ask'
            else:
                time.sleep(5)
        else:
            if bid > exchange.get_last_price_book(stock_id).bids[0].price and exchange.get_positions()[STOCK_A_ID] + exchange.get_positions()[STOCK_B_ID] <= 10:
                side = 'bid'
            else:
                time.sleep(5)
            
    print(f'Selected stock {stock_id} to trade.')

    # Obtain order book and only trade if there are both bids and offers available on that stock
    stock_order_book = exchange.get_last_price_book(stock_id)
    if not (stock_order_book and stock_order_book.bids and stock_order_book.asks):
        print(f'Order book for {stock_id} does not have bids or offers. Skipping iteration.')
        continue

    # Obtain best bid and ask prices from order book
    best_bid_price = stock_order_book.bids[0].price
    best_ask_price = stock_order_book.asks[0].price
    print(f'Top level prices for {stock_id}: {best_bid_price:.2f} :: {best_ask_price:.2f}')

    if stock_id == STOCK_A_ID:
        if side == 'bid':
            price = best_bid_price
            bida = price
        else:
            price = best_ask_price
            aska = price
    if stock_id == STOCK_B_ID:
        if side == 'bid':
            price = best_bid_price
            bidb = price
        else:
            price = best_ask_price
            askb = price

    # Insert an IOC order to trade the opposing top-level, ensure to always keep instrument position below 5 so
    # aggregate position stays below 10.
    volume = 5
    if not trade_would_breach_position_limit(stock_id, volume, side, 5):
        print(f'''Inserting {side} for {stock_id}: {volume:.0f} lot(s) at price {price:.2f}.''')
        exchange.insert_order(
            instrument_id=stock_id,
            price=price,
            volume=volume,
            side=side,
            order_type='ioc')
    else:
        print(f'''Not inserting {volume:.0f} lot {side} for {stock_id} to avoid position-limit breach.''')

    print(f'\nSleeping for 3 seconds.')
    time.sleep(3)